Textbook RSA Dummy Implementation
===================

This file is an implementation of the scheme invented by Rivest, Shamir and Adleman.

__Note for using Jupyter__: Use $\texttt{SHIFT + ENTER}$ to execute the code snippets!

Recall the __RSA experiment__: 

Given security parameter $\lambda = 1^n$ a PPT adversary $\mathcal{A}$ and PPT algorithm $\texttt{GenRSA}$, the RSA experiment is as follows
1. Run $(N, e, d) \leftarrow \texttt{GenRSA}(\lambda)$.
2. Uniformly pick $y \leftarrow \mathbb{Z}^*_N$.
3. Run $x = \mathcal{A}(N, e, y)$ where $x \in \mathbb{Z}^*_N$.
4. Set $\texttt{Expr}^{\text{RSA}}_{\mathcal{A}, \texttt{GenRSA}}(\lambda) = 1$ if $x^e = y$ and $0$ otherwise.

Informally, the RSA experiment is merely asking the adversary to take the $e$th root of $y$ modulo $N$. Now, for the hardness assumption:

__RSA hardness assumption__:

The RSA experiment is hard relative to $\texttt{GenRSA}$ if every PPT adversary $\mathcal{A}$ has negligible advantage of succeeding to solve the experiment:
$$
\texttt{Adv}^{\text{RSA}}_{\mathcal{A}, \texttt{GenRSA}}(\lambda) = \mathbb{P}[\texttt{Expr}^{\text{RSA}}_{\mathcal{A}, \texttt{GenRSA}}(\lambda) = 1] \leq \text{negl}(n).
$$

Recall, that it is conjectured that if $N = pq$ for some $p, q = \mathcal{O}(\sqrt{N})$ primes and arbitrary exponent $e$ the RSA experiment is indeed very hard to solve (it is NP-hard). We can then use this to create a trapdoor and based on that a crypto scheme, that we will see below.

In [2]:
load('./rsa.sage')

Key Generation
-----------------------------------------

We begin with key generation. Given a security parameter $\lambda = 1^n$ we perform the following:

1. Pick two different primes $|p| = |q|$ (here $|\cdot|$ measures the length of the binary representation of numbers).
2. Calculate $N = pq$ and $\phi(N) = (p - 1)(q - 1)$.
3. Pick an element $e$ such that $\gcd(e, \phi(N)) = 1$. This means that $e \in \mathbb{Z}^*_N$, and hence invertible mod $N$.
4. Output $pk = (N, e)$ and $sk = (N, e^{-1})$

Below we have performed this procedure precisely:

In [3]:
pk, sk = key_gen(50);pk, sk

((3654282358540037353184490864391, 1600886757123181629156101005139),
 (3654282358540037353184490864391, 1744501367292958863880289016059))

Encryption
---------------------------------

Encryption is rather simple, we are just following the RSA experiment:

Given public key $pk = (N, e)$ and message $m$, output $c = m^e \mod N$.



In [4]:
m = 1234
c = encrypt(pk, m);c

3053535034713184197374850046361

Decryption
---------------------------------------

Decryption will make use of our trapdoor: when we are only given $N, e$ and $x^e$ for some $x$, it is hard to invert the encryption function. However, leveraging the fact that $e$ is invertible in $\mathbb{Z}^*_N$ and in any group $G$ we have $g^p = g^{p \mod |G|}$ for arbitrary $p \in \mathbb{Z}$, given a secret key $sk = (N, d = e^{-1})$ and ciphertext $c$, we can calculate and output
$$
    m' = c^d\mod N.
$$

Correctness
----------------------------------------

Combining the encryption and decryption functions then, given $pk = (N, e)$, $sk = (N, d = e^{-1})$ and message $m \in \mathbb{Z}_N$, we have


$$ 
    m' = \text{Dec}_{sk}(\text{Enc}_{pk}(m))
$$


$$
    m' = (m^e\mod N)^d\mod N
$$

$$
    m' = m^{ed\mod N}\mod N
$$
Then, $ed = ee^{-1} = 1 \mod N$ by the definition of the cipher and hence
$$
    m' = m^{1 \mod N}\mod N = m.
$$

Below, we see can test the correctness in practice:

In [5]:
m_prime = decrypt(sk, c);m_prime

1234

Notes about the security of this scheme
----------------------------------------------

Recall from Section 5.6 in the report that since this encryption function is deterministic, thes scheme cannot be IND-CPA secure.

We can actually expose an adversary $\mathcal{A}$ that breaks this security level quite easily:

In [6]:
def A(pk, challenge=None, state=None):
    
    (N, e) = pk
    
    # If we don't pass a challenge, we are in stage 1
    if challenge is None:
        
        # We pick two arbitrary different messages
        # from the message space
        m0 = ZZ.random_element(N)
        
        m1 = -1
        while m0 == m1:
            m1 = ZZ.random_element(N)
            
        # We precompute the eth power of m0 mod N
        state = mod(m0, N) ^ e
        
        return (m0, m1), state
    
    else:
        
        return 0 if challenge == state else 1

Then, we can define the IND-CPA security experiment:

In [7]:
def Expr(sec_param, Pi, A):
    """
    IND-CPA security experiment.
    
    We take a security parameter lambda, 
    a cipher Pi and PPT adversary A
    """
    (gen, enc, dec) = Pi
    
    # Step 1: generate key pair
    (pk, sk) = gen(sec_param)
    
    # Step 2: obtain messages from the adversary
    (m0, m1), state = A(pk)
    
    # Step 3: generate a uniform bit and calculate the challenge
    b = ZZ.random_element(2)
    
    challenge = enc(pk, m0) if b == 0 else enc(pk, m1)
    
    # Step 4: see if the adversary succeeds
    return 1 if A(pk, challenge=challenge, state=state) == b else 0

Now, let's see if $\mathcal{A}$ indeed succeeds with non-negligible advantage:

In [8]:
Pi = (key_gen, encrypt, decrypt)

sec_param = 20

counter = 0;

for i in range(100):
    # Count the number of times the adversary solves the experiment
    counter += Expr(sec_param, Pi, A)
    
counter    

100

We see that the above adversary not only has non-negligble advantage, but breaks the scheme with probability 1! 

In particular, this is the special case of Theorem 5.4 for RSA.

Secondly, we note that the security parameter must be high enough, since we are relying on the fact that it is hard to factorize $N$. We illustrate this below:

In [9]:
factors = map(lambda tup: tup[0], list(factor(pk[0])));factors

[1705767464357521, 2142309801832471]

We can now calculate $\phi(N)$, and use it to invert the exponent and break the scheme:

In [10]:
phi = (factors[0] - 1) * (factors[1] - 1)

(_, e_inv, _) = xgcd(pk[1], phi)

broken_sk = (pk[0], e_inv)

decrypt(broken_sk, c)

1234